In [ ]:
import pandas as pd
import xlwings as xw
import openpyxl
from openpyxl.utils import get_column_letter
from tqdm.notebook import tqdm

In [ ]:
# Path to template sheet for use in Insert sheet
template_path = 'tracker_template.xlsx'
# Path to workbook with table of Workbook Names and Sheet Names
names_wb_path = 'names.xlsx'
names_wb = openpyxl.load_workbook(names_wb_path)
# Select correct sheet
names_sheet = names_wb['Sheet1']
# Parse as df
df = pd.DataFrame(names_sheet.values)
# Set headers
df.columns = df.iloc[0].values
# Drop row with header names
df = df.drop(0)

In [ ]:
for index, row in df.iterrows():
    df['Sheet Name'].loc[index] = row['Sheet Name'][:30]

In [ ]:
def Book_Gen(df):
    '''This function will create workbooks from the
    dataframe of workbook names provided.
    There will be a workbook created for each unique value in the Workbook Name
    column of df.
    inputs:
        df = the dataframe housing Workbook Name and Sheet Name
    outputs:
        prints name of each workbook created
        creates workbooks in script location'''
    
    workbooks = {}
    
    # Iterate through df and create wb and sheets
    for index, row in tqdm(df.iterrows()):
        workbook_name = row['Workbook Name'] + '.xlsx'
        # Check for existing workbook
        if workbook_name not in workbooks:
            workbooks[workbook_name] = openpyxl.Workbook()
            #print(f'{workbook_name} Created')
            
    # Close all workbooks
    for workbook_name, workbook in workbooks.items():
        workbook.save(workbook_name)

In [ ]:
def Insert_Sheet(template_path, df):
    '''Inserts the template_sheet for each Sheet Name into each Workbook Name in df
    inputs:
        df = the dataframe housing Workbook Name and Sheet Name
    outputs:
        prints name of each sheet created
        inserts sheets into workbooks in script location'''
    
    # Open template sheet
    template_workbook = xw.Book(template_path)
    template_sheet = template_workbook.sheets[0]
    
    # Dict of workbooks
    workbooks = {}
    # List of workbook names
    workbook_names = list(df['Workbook Name'].value_counts().index)
    
    for workbook_name in tqdm(workbook_names, desc='Outer Loop'):
        workbook_filename = workbook_name + '.xlsx'
        workbook = xw.Book(workbook_filename)
        
        for index, row in tqdm(df[df['Workbook Name'] == workbook_name].iterrows(), desc='Inner Loop', position=1):
            sheet_name = row['Sheet Name']
            template_sheet.copy(before=workbook.sheets[0], name=sheet_name)
            #tqdm.write(f'{sheet_name} added to {workbook_name}')
            
        workbook.sheets['Sheet'].delete()
        workbook.save()
        workbook.close()
        template_workbook.close()
        #tqdm.write(f'{workbook_filename} Complete')


In [ ]:
# Generates a new line in the rollup sheet for a proposal
def lineGen(proposal_name, book_name, index, df):
    line = [proposal_name]
    formulas = [f'''=INDIRECT("'{proposal_name}'!G{x}")''' for x in range(6,19)]
    formulas.insert(0, f'{proposal_name}')
    formulas.insert(1, f'''=sum(C{index+2}:O{index+2})''')
    df.loc[len(df)] = formulas

In [ ]:
Book_Gen(df)

In [ ]:
Insert_Sheet(template_path, df)